In [72]:
import pandas as pd
import os
print(os.getcwd())
import pandas as pd
print(pd.__version__)
!python --version
!jupyter --version

# Load data from the input folder
enrollment_data = pd.read_csv("../data/input/CPSC_Enrollment_2015_01/CPSC_Enrollment_Info_2015_01.csv")
service_area_data = pd.read_csv("../data/input/MA_Cnty_SA_2015_01/MA_Cnty_SA_2015_01.csv")
contract_data = pd.read_csv("../data/input/CPSC_Enrollment_2015_01/CPSC_Contract_Info_2015_01.csv", encoding='latin1')
pd.set_option('display.max_columns', None)



# Display the datasets
print(enrollment_data.head())
print(service_area_data.head())


/Users/ryanscholte/Desktop/GitHub/HW1/submission1
2.2.3
Python 3.10.16
Selected Jupyter core packages...
IPython          : 8.30.0
ipykernel        : 6.29.5
ipywidgets       : not installed
jupyter_client   : 7.4.9
jupyter_core     : 5.7.2
jupyter_server   : 2.7.0
jupyterlab       : 4.3.4
nbclient         : 0.10.2
nbconvert        : 7.16.5
nbformat         : 5.10.4
notebook         : 7.3.2
qtconsole        : not installed
traitlets        : 5.14.3
  Contract Number  Plan ID  SSA State County Code  FIPS State County Code  \
0           E0654      801                  58160                     NaN   
1           E0654      801                  61000                     NaN   
2           E0654      801                  96001                     NaN   
3           E0654      801                   2275                     NaN   
4           E0654      801                   2198                     NaN   

  State County Enrollment  
0   NaN    NaN          *  
1   NaN    NaN          *  
2

/var/folders/mn/l2nrwsxn24g6ywz6ygh2fxp40000gn/T/ipykernel_3527/4202740048.py:11: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  service_area_data = pd.read_csv("../data/input/MA_Cnty_SA_2015_01/MA_Cnty_SA_2015_01.csv")


In [73]:
print(enrollment_data.columns)
print(contract_data.columns)
print(service_area_data.columns)


Index(['Contract Number', 'Plan ID', 'SSA State County Code',
       'FIPS State County Code', 'State', 'County', 'Enrollment'],
      dtype='object')
Index(['Contract ID', 'Plan ID', 'Organization Type', 'Plan Type',
       'Offers Part D', 'SNP Plan', 'EGHP', 'Organization Name',
       'Organization Marketing Name', 'Plan Name', 'Parent Organization',
       'Contract Effective Date'],
      dtype='object')
Index(['Contract ID', 'Organization Name', 'Organization Type', 'Plan Type',
       'Partial', 'EGHP', 'SSA', 'FIPS', 'County', 'State', 'Notes'],
      dtype='object')


In [74]:
#rename Contract Number to Contract ID , SSA State county code to SSA, and FIPS state county code to FIPS in enrollment data
enrollment_data.rename(columns={'Contract Number': 'Contract ID', 'SSA State County Code': 'SSA', 'FIPS State County Code': 'FIPS'}, inplace=True)

 # merge enrollment data with contract data on 'Contract ID' and 'Plan ID'
merged_data = pd.merge(enrollment_data, contract_data, on=['Contract ID', 'Plan ID'], how='left')

# #show me the merged data head
print(merged_data.head())

  Contract ID  Plan ID    SSA  FIPS State County Enrollment  \
0       E0654      801  58160   NaN   NaN    NaN          *   
1       E0654      801  61000   NaN   NaN    NaN          *   
2       E0654      801  96001   NaN   NaN    NaN          *   
3       E0654      801   2275   NaN   NaN    NaN          *   
4       E0654      801   2198   NaN   NaN    NaN          *   

                         Organization Type  \
0  Employer/Union Only Direct Contract PDP   
1  Employer/Union Only Direct Contract PDP   
2  Employer/Union Only Direct Contract PDP   
3  Employer/Union Only Direct Contract PDP   
4  Employer/Union Only Direct Contract PDP   

                                 Plan Type Offers Part D SNP Plan EGHP  \
0  Employer/Union Only Direct Contract PDP           Yes       No  Yes   
1  Employer/Union Only Direct Contract PDP           Yes       No  Yes   
2  Employer/Union Only Direct Contract PDP           Yes       No  Yes   
3  Employer/Union Only Direct Contract PDP      

In [75]:
# Convert Plan ID to same type in both dataframes
enrollment_data['Plan ID'] = enrollment_data['Plan ID'].astype(float)
contract_data['Plan ID'] = contract_data['Plan ID'].astype(float)
merged_data['Plan ID'] = merged_data['Plan ID'].astype(float)
# Convert Contract ID to string type in both
enrollment_data['Contract ID'] = enrollment_data['Contract ID'].astype(str)
contract_data['Contract ID'] = contract_data['Contract ID'].astype(str)
merged_data['Contract ID'] = merged_data['Contract ID'].astype(str)

# Merge 
merged_all = pd.merge(merged_data, 
                                   service_area_data,
                                   on=['Contract ID', 'SSA', 'FIPS', 'State', 'County'],
                                   how='inner')

# check the merged data
print(merged_all.head())

  Contract ID  Plan ID    SSA     FIPS State    County Enrollment  \
0       H0022      1.0  36110  39023.0    OH     Clark        598   
1       H0022      1.0  36170  39035.0    OH  Cuyahoga       2810   
2       H0022      1.0  36260  39051.0    OH    Fulton        112   
3       H0022      1.0  36280  39055.0    OH    Geauga         48   
4       H0022      1.0  36290  39057.0    OH    Greene        528   

  Organization Type_x                        Plan Type_x Offers Part D  \
0                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
1                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
2                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
3                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
4                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   

  SNP Plan EGHP_x                  Organization Name_x  \
0       No     No  BUCKEYE COMMUNITY HEALTH PLAN, INC.   
1       No     No  BUCKE

In [76]:
# count how many duplicate rows are in the merged data
duplicates = merged_all.duplicated()
print(duplicates.sum())

# count duplicates in contract data and enrollment data and service area data
duplicates_contract = contract_data.duplicated()
duplicates_enrollment = enrollment_data.duplicated()
duplicates_service_area = service_area_data.duplicated()
print(duplicates_contract.sum())
print(duplicates_enrollment.sum())
print(duplicates_service_area.sum())

# # Create new id column in merged data that takes the Plan ID and Contract ID and makes a unique id for each combination of the two
# merged_all['id'] = merged_all['Plan ID'].astype(str) + '-' + merged_all['Contract ID'].astype(str)
# #check duplicates in the new id column
# duplicates_id = merged_all.duplicated(subset='id')
# print(duplicates_id.sum())
# #drop duplicates
# merged_all.drop_duplicates(subset='id', inplace=True)
# #check duplicates again
# duplicates_id = merged_all.duplicated(subset='id')
# print(duplicates_id.sum())


0
0
0
0


In [77]:
#rename column "Plan Type_x" to "Plan Type", EGHP_x to EGHP
merged_all.rename(columns={'Plan Type_x': 'Plan Type', 'EGHP_x': 'EGHP'}, inplace=True)

In [78]:
# fill in missing FIPS values with known FIPS values matching the county and state found elsewhere in the dataset
merged_all['FIPS'] = merged_all['FIPS'].fillna(merged_all.groupby(['State', 'County'])['FIPS'].transform('first'))
# fill in missing 'Offers Part D', 'SNP Plan', 'EGHP', values with known values matching from the Plan Type found elsewhere in the dataset
merged_all['Offers Part D'] = merged_all['Offers Part D'].fillna(merged_all.groupby(['Plan Type'])['Offers Part D'].transform('first'))
merged_all['SNP Plan'] = merged_all['SNP Plan'].fillna(merged_all.groupby(['Plan Type'])['SNP Plan'].transform('first'))
merged_all['EGHP'] = merged_all['EGHP'].fillna(merged_all.groupby(['Plan Type'])['EGHP'].transform('first'))


In [79]:
# Count the number of plans for each plan type
plan_counts = merged_all.groupby('Plan Type').size().reset_index(name='Plan Count')
plan_counts.style.format({'Plan Count': '{:,.0f}'}).hide(axis='index')

Plan Type,Plan Count
1876 Cost,"5,527"
HMO/HMOPOS,"453,802"
Local PPO,"701,380"
MSA,"6,502"
Medicare-Medicaid Plan HMO/HMOPOS,859
National PACE,649
PFFS,"12,596"
Regional PPO,"16,048"


In [80]:
#make merged data 2 set which drops all special needs plans (SNP), employer group plans (eghp), and all “800-series”
# in "SNP Plan" column drop is says yes, in "EGHP" coloumn if says yes drop it, in "Plan ID" column between 800-900 drop it
merged_all2 = merged_all[~merged_all['SNP Plan'].str.contains('Yes', na=False) &
                           ~merged_all['EGHP'].str.contains('Yes', na=False) &
                           ~merged_all['Plan ID'].between(800, 900, inclusive="left")]
print(merged_all2.head())

  Contract ID  Plan ID    SSA     FIPS State    County Enrollment  \
0       H0022      1.0  36110  39023.0    OH     Clark        598   
1       H0022      1.0  36170  39035.0    OH  Cuyahoga       2810   
2       H0022      1.0  36260  39051.0    OH    Fulton        112   
3       H0022      1.0  36280  39055.0    OH    Geauga         48   
4       H0022      1.0  36290  39057.0    OH    Greene        528   

  Organization Type_x                          Plan Type Offers Part D  \
0                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
1                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
2                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
3                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
4                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   

  SNP Plan EGHP                  Organization Name_x  \
0       No   No  BUCKEYE COMMUNITY HEALTH PLAN, INC.   
1       No   No  BUCKEYE COM

In [81]:
# Count the number of plans for each plan type for merged all 2 
plan_counts2 = merged_all2.groupby('Plan Type').size().reset_index(name='Plan Count')
plan_counts2.style.format({'Plan Count': '{:,.0f}'}).hide(axis='index')

Plan Type,Plan Count
1876 Cost,"4,905"
HMO/HMOPOS,"19,449"
Local PPO,"14,618"
MSA,222
Medicare-Medicaid Plan HMO/HMOPOS,859
National PACE,649
PFFS,"3,179"
Regional PPO,"7,375"


In [82]:
#show values for enrollment column in merged all 2
print(merged_all2['Enrollment'].value_counts())
# sum of enrollment column in merged all 2
# print(merged_all2['Enrollment'].sum())




Enrollment
*       26511
11        439
13        436
12        430
14        409
        ...  
3081        1
5425        1
5543        1
3928        1
1089        1
Name: count, Length: 2637, dtype: int64


In [83]:
#remove non numeric values from enrollment column
merged_all2['Enrollment'] = pd.to_numeric(merged_all2['Enrollment'], errors='coerce')
#average enrollments for each plan type
average_enrollment = merged_all2.groupby('Plan Type')['Enrollment'].mean().reset_index(name='Average Enrollment')
average_enrollment.style.format({'Average Enrollment': '{:,.3f}'}).hide(axis='index')

/var/folders/mn/l2nrwsxn24g6ywz6ygh2fxp40000gn/T/ipykernel_3527/3685670825.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_all2['Enrollment'] = pd.to_numeric(merged_all2['Enrollment'], errors='coerce')


Plan Type,Average Enrollment
1876 Cost,229.650
HMO/HMOPOS,871.225
Local PPO,311.063
MSA,107.793
Medicare-Medicaid Plan HMO/HMOPOS,753.030
National PACE,144.488
PFFS,125.366
Regional PPO,201.503
